# Models

In [1]:
#usual data manipluation/plot tools
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#ML tools
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


C:\Users\jc243891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#create dataype dictionary for data, then import our cleaned data.
datatypes = {'Primary': 'category', 'UMSCUMCC': 'category', 'Age_at_dx': 'float', 'Sex': 'category',
       'Marital_status': 'category', 'Meno_stat': 'category', 'famhist': 'category', 'Nationality': 'category', 'Race': 'category',
       'diagnosis': 'category', 'Classification': 'category', 'Laterality': 'category', 'STAGEAJCC6': 'category',
       'Differentiation': 'category', 'ER_status': 'category', 'PR_status': 'category', 'CERB2_status': 'category', 'Pri_Rx': 'category',
       'Surgery_type': 'category', 'Axill_dissect': 'category', 'Radiotherapy': 'category', 'Chemotherapy': 'category',
       'Event': 'category'}

data = pd.read_csv('../../clean_cancer_data.csv', sep='\t',dtype=datatypes).copy()
data.rename(index=str, columns={'Unnamed: 0':'Patient_num'},inplace=True)


In [3]:
#all looks good
data.info()
data.Duration


<class 'pandas.core.frame.DataFrame'>
Index: 2547 entries, 0 to 2546
Data columns (total 25 columns):
Patient_num        2547 non-null int64
Primary            2547 non-null category
UMSCUMCC           2547 non-null category
Age_at_dx          2547 non-null float64
Sex                2547 non-null category
Marital_status     2547 non-null category
Meno_stat          2547 non-null category
famhist            2547 non-null category
Nationality        2547 non-null category
Race               2547 non-null category
diagnosis          2547 non-null category
Classification     2547 non-null category
Laterality         2547 non-null category
STAGEAJCC6         2547 non-null category
Differentiation    2547 non-null category
ER_status          2547 non-null category
PR_status          2547 non-null category
CERB2_status       2547 non-null category
Pri_Rx             2547 non-null category
Surgery_type       2547 non-null category
Axill_dissect      2547 non-null category
Radiotherapy       2

0        240.0
1        730.0
2         12.0
3        365.0
4        120.0
5        182.5
6         12.0
7         12.0
8        150.0
9        180.0
10       120.0
11      1095.0
12        12.0
13        24.0
14       210.0
15       182.5
16       180.0
17       730.0
18       180.0
19      1095.0
20       730.0
21       365.0
22       182.5
23       182.5
24       210.0
25        12.0
26       730.0
27      1095.0
28       182.5
29       180.0
         ...  
2517      90.0
2518    1095.0
2519     365.0
2520      90.0
2521      90.0
2522     365.0
2523      21.0
2524      60.0
2525      12.0
2526     120.0
2527      13.0
2528      30.0
2529     730.0
2530     365.0
2531     384.0
2532      60.0
2533     120.0
2534       0.0
2535       0.0
2536     730.0
2537     365.0
2538       0.0
2539     365.0
2540       0.0
2541       0.0
2542     365.0
2543    1095.0
2544     120.0
2545     365.0
2546       0.0
Name: Duration, Length: 2547, dtype: float64

## Preprocessing
Not a lot do do here, we just need to normalise the Duration column and split the data into a training and testing set (train/test: 80/20)

In [16]:
from sklearn import preprocessing
proc_data = data[['Primary','UMSCUMCC','Age_at_dx','Sex','Marital_status','Meno_stat','famhist','Nationality','Race','diagnosis','Classification','Laterality','STAGEAJCC6','Differentiation','ER_status','PR_status','CERB2_status','Pri_Rx','Surgery_type','Axill_dissect','Radiotherapy','Chemotherapy','Duration']].copy()
duration = proc_data[['Duration']].values.astype(float)
## Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()
# Create an object to transform the data to fit minmax processor
duration_scaled = min_max_scaler.fit_transform(duration)
# Run the normalizer on the dataframe
proc_data.Duration = duration_scaled


In [17]:
cancer_target = data[['Event']].copy()
print(data.Event.value_counts())

X_train, X_test, y_train, y_test = train_test_split(proc_data, np.ravel(cancer_target), test_size=0.20, random_state=101)

0    2068
1     479
Name: Event, dtype: int64


## SVM

In [18]:
#create and train model
model = SVC()
model.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [19]:
#check precision using the test data (x_test and y_test)
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.81      1.00      0.90       414
          1       0.00      0.00      0.00        96

avg / total       0.66      0.81      0.73       510



C:\Users\jc243891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Obviously this result is not good, therefore we need to restructure the data. Everything but Duration is a category, so let's convert this to categorical data. First we need to get rid of the normalised data, then we need to create a set of categories using cut.

In [20]:
proc_data['Duration'] = pd.cut(data.Duration.copy()/365,[-1,0,1,2,3,4,5,100],labels=["0", "0-1", "1-2","2-3","3-4","4-5","5<"])
proc_data.Duration.value_counts()


0-1    1813
0       282
1-2     237
2-3     206
5<        5
4-5       3
3-4       1
Name: Duration, dtype: int64

There is very few values for over 2 years so we can just redo and combine this as the category '2<', we will also put in categories for "0-0.25" and "0.25-0.5", so that all of our data is not just concentrated into the first bin, our categories will essentially be: ["0yrs","0-0.25yrs","0.25-0.5yrs", "0.5-1yrs", "1-2yrs","2<yrs"]

In [9]:
proc_data['Duration'] = pd.cut(data.Duration.copy()/365,[-1,0,0.25,0.5,1,2,100],labels=['0','1','2','3','4','5']) #check how many people die at 0-1
proc_data.head()

,Primary,UMSCUMCC,Age_at_dx,Sex,Marital_status,Meno_stat,famhist,Nationality,Race,diagnosis,...,Differentiation,ER_status,PR_status,CERB2_status,Pri_Rx,Surgery_type,Axill_dissect,Radiotherapy,Chemotherapy,Duration
0,0,1,33.0,0,0,1,0,0,0,0,...,1,1,1,1,1,0,0,0,0,3
1,0,0,60.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,0,0,65.0,0,0,0,1,0,0,0,...,1,0,0,2,0,0,0,0,0,1
3,0,0,34.0,0,0,1,0,0,2,2,...,1,1,1,0,0,0,0,0,0,3
4,0,0,47.0,0,0,1,0,0,1,2,...,1,0,1,0,0,0,0,1,0,2


In [10]:
#OK lets try again
X_train, X_test, y_train, y_test = train_test_split(proc_data, np.ravel(cancer_target), test_size=0.20,random_state=101)
model = SVC()
model.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [11]:
#check precision using the test data (x_test and y_test)
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.81      1.00      0.90       414
          1       0.00      0.00      0.00        96

avg / total       0.66      0.81      0.73       510



C:\Users\jc243891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### SAME ERROR, WHAT IS GOING WRONG! I EVEN GET THE ERROR WHEN DURATION IS TAKEN OUT
could this be related to the nan values we took out? does this reduce the dataset too much?

In [12]:

param_grid = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001], 'kernel':['rbf']}

from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)

grid.fit(X_train, y_train)

print('\n')
print('The best parameters are ', grid.best_params_)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV]  C=0.1, gamma=1, kernel=rbf, score=0.8117647058823529, total=   0.1s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  C=0.1, gamma=1, kernel=rbf, score=0.8114874815905744, total=   0.1s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV]  C=0.1, gamma=1, kernel=rbf, score=0.8126843657817109, total=   0.1s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV]  C=0.1, gamma=0.1, kernel=rbf, score=0.8117647058823529, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV]  C=0.1, gamma=0.1, kernel=rbf, score=0.8114874815905744, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV]  C=0.1, gamma=0.1, kernel=rbf, score=0.8126843657817109, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV]  C=0.1, gamma=0.01, kernel=rbf, score=0.8117647058823529, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV]  C=0.1, gamma=0.01, kernel=rbf, score=0.8114874815905744, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV]  C=0.1, gamma=0.01, kernel=rbf, score=0.8126843657817109, total=   0.0s
[CV] C=0.1, gamma=0.001, kernel=rbf .....

[CV]  C=100, gamma=0.0001, kernel=rbf, score=0.8114874815905744, total=   0.1s
[CV] C=100, gamma=0.0001, kernel=rbf .................................
[CV]  C=100, gamma=0.0001, kernel=rbf, score=0.8126843657817109, total=   0.1s
[CV] C=1000, gamma=1, kernel=rbf .....................................
[CV]  C=1000, gamma=1, kernel=rbf, score=0.8117647058823529, total=   0.1s
[CV] C=1000, gamma=1, kernel=rbf .....................................
[CV]  C=1000, gamma=1, kernel=rbf, score=0.8114874815905744, total=   0.1s
[CV] C=1000, gamma=1, kernel=rbf .....................................
[CV]  C=1000, gamma=1, kernel=rbf, score=0.8112094395280236, total=   0.1s
[CV] C=1000, gamma=0.1, kernel=rbf ...................................
[CV]  C=1000, gamma=0.1, kernel=rbf, score=0.7602941176470588, total=   0.1s
[CV] C=1000, gamma=0.1, kernel=rbf ...................................
[CV]  C=1000, gamma=0.1, kernel=rbf, score=0.7982326951399117, total=   0.1s
[CV] C=1000, gamma=0.1, kernel=rbf ..

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:   17.2s finished


In [13]:
grid_predictions = grid.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, grid_predictions))

             precision    recall  f1-score   support

          0       0.81      0.99      0.89       414
          1       0.33      0.02      0.04        96

avg / total       0.72      0.81      0.73       510

